In [ ]:
import gradio as gr
import requests
from datetime import datetime
from openai import AzureOpenAI
import traceback


# Azure OpenAI 설정
client = AzureOpenAI(
    api_key="",
    azure_endpoint="https://6b013-azure-ai-service.openai.azure.com/",
    api_version="2024-05-01-preview"
)
AZURE_DEPLOYMENT_NAME = "gpt-4o"

# 카드 업데이트
def update_cards(title_text):
    return title_text, title_text, title_text

# 오답 데이터 불러오기

def fetch_wrong_answers_cards(user_id=1):
    try:
        res = requests.get("http://localhost:8000/get-wrong-answers", params={"user_id": user_id})
        data = res.json()["wrong_answers"]

        if not data:
            return "✅ 오답이 없습니다!"

        cards = []
        for idx, item in enumerate(data, 1):
            q = f"**{idx}. 문제:** {item['question_text']}"
            choices = "\n".join([
                f"   - ① {item['choice1']}",
                f"   - ② {item['choice2']}",
                f"   - ③ {item['choice3']}",
                f"   - ④ {item['choice4']}"
            ])
            answer = f"✅ 정답: {item['answer']}번"
            my_choice = f"❌ 내 선택: {item['user_choice']}번"
            exp = f"📘 해설: {item['explanation']}"
            cards.append(f"{q}\n{choices}\n{answer} | {my_choice}\n{exp}")

        return "\n\n---\n\n".join(cards)
    except Exception as e:
        return f"❗ 오류: {str(e)}"

def update_wrong_note_textbox():
    text = fetch_wrong_answers_cards()
    return gr.update(value=text)


# 문제 생성

def add_chat(user_msg, history):
    if history is None:
        history = []
    history.append((user_msg, None))

    system_prompt = """
너는 한국사능력검정시험 문제를 생성하는 AI 튜터야.
절대로 아래 형식을 벗어나지 마. 다른 문장 추가 금지.

형식 예시:
문제: 백제의 수도가 한성에서 웅진으로 옮겨진 이유는?
보기:
1. 신라와의 동맹
2. 고구려의 침입
3. 내부 반란
4. 일본과의 관계 악화
정답: 2
해설: 475년 고구려 장수왕의 공격으로 백제 수도 한성이 함락되었고, 문주왕은 웅진으로 천도하였다.
"""

    messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_msg}
    ]

    try:
        completion = client.chat.completions.create(
            model=AZURE_DEPLOYMENT_NAME,
            messages=messages,
            max_tokens=800,
            temperature=0.7
        )

        result = completion.choices[0].message.content
        print("📦 GPT 응답 원문:\n", result)

        if all(k in result for k in ["문제:", "보기:", "정답:", "해설:"]):
            parts = result.split("문제:")[1].strip().split("보기:")
            question = parts[0].strip()
            choice_lines = parts[1].split("정답")[0].strip().split("\n")
            choices = [line.strip() for line in choice_lines if line.strip()]
            answer = int(result.split("정답:")[1].split("\n")[0].strip())
            explanation = result.split("해설:")[1].strip()
        else:
            raise ValueError("GPT 응답 형식이 올바르지 않아요.")

        save_res = requests.post("http://localhost:8000/save-question", data={
            "material_id": 12,
            "question_text": question,
            "choice1": choices[0],
            "choice2": choices[1],
            "choice3": choices[2],
            "choice4": choices[3],
            "answer": answer,
            "explanation": explanation
        })
        question_id = save_res.json().get("question_id", )

        ai_text = question + "\n" + "\n".join(choices)
        history.append((None, ai_text))

        ai_response = {
            "문제": question,
            "보기": choices,
            "정답": answer - 1,
            "해설": explanation,
            "question_id": question_id
        }

    except Exception as e:
        print("🔴 오류:", traceback.format_exc())
        history.append((None, f"❌ 문제 생성 실패: {str(e)}"))
        ai_response = {"정답": 0, "해설": "N/A", "question_id": 0}

    return history, ai_response

# 정답 체크 및 오답 저장 포함
def check_answer(user_choice, ai_response, history):
    if history is None:
        history = []

    correct = ai_response["정답"] + 1
    explanation = ai_response["해설"]
    result = "✅ 정답입니다!" if int(user_choice) == correct else f"❌ 오답입니다. 정답은 {correct}번이에요."

    history.append((f"정답: {user_choice}번", None))
    history.append((None, f"{result}\n📘 해설: {explanation}"))

    if int(user_choice) != correct:
        try:
            requests.post("http://localhost:8000/save_wrong_answer", data={
                "user_id": 1,
                "question_id": ai_response["question_id"],
                "user_choice": int(user_choice)
            })
        except Exception as e:
            print("❗ 오답 저장 실패:", e)

    return history

# 전체 UI
def build_ui():
    with gr.Blocks(title="한국사능력검정시험 AI 학습") as demo:
        with gr.Tabs():
            with gr.TabItem("AI 문제"):
                gr.Markdown("### 💬 AI 문제 - RAG 기반 GPT-4o 문제 생성")
                chatbot = gr.Chatbot(label="한국사 문제 챗봇", height=450)
                state = gr.State([])
                ai_state = gr.State({})

                with gr.Row():
                    user_input = gr.Textbox(placeholder="예: 백제 문화 관련 문제 내줘", label="문제 요청", scale=8)
                    send_btn = gr.Button("요청", scale=2)

                answer_dropdown = gr.Radio(choices=["1", "2", "3", "4"], label="정답 선택", visible=False)
                submit_answer = gr.Button("정답 제출", visible=False)

                send_btn.click(fn=add_chat, inputs=[user_input, state], outputs=[chatbot, ai_state])
                send_btn.click(lambda: gr.update(visible=True), outputs=answer_dropdown)
                send_btn.click(lambda: gr.update(visible=True), outputs=submit_answer)
                submit_answer.click(fn=check_answer, inputs=[answer_dropdown, ai_state, state], outputs=chatbot)

            with gr.TabItem("📕 오답 노트"):
                gr.Markdown("## 📕 나의 오답 노트 (리스트 보기)")
                wrong_note_output = gr.Textbox(lines=20, interactive=False, label="오답 목록", show_copy_button=True)
                refresh_btn = gr.Button("🔁 오답 다시 불러오기")
                refresh_btn.click(fn=update_wrong_note_textbox, inputs=[], outputs=[wrong_note_output])

    return demo

demo = build_ui()
demo.launch()


/var/folders/sh/pq4glsrs3xq2v8ymsrw74mrr0000gn/T/ipykernel_91386/3970136668.py:195: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style dictionaries with 'role' and 'content' keys.
  chatbot = gr.Chatbot(label="한국사 문제 챗봇", height=450)


* Running on local URL:  http://127.0.0.1:7862

To create a public link, set `share=True` in `launch()`.


📦 GPT 응답 원문:
 문제: 백제의 전성기를 이끌었던 왕으로, 황해도 지역을 점령하고 중국 남조와 활발히 교류한 왕은 누구인가?  
보기:  
1. 고이왕  
2. 근초고왕  
3. 성왕  
4. 무왕  
정답: 2  
해설: 근초고왕(4세기)은 백제의 전성기를 이끈 왕으로, 황해도 지역을 점령하며 영토를 확장했고, 중국 남조와 교류를 통해 백제의 국제적 위상을 높였다.  
🔴 오류: Traceback (most recent call last):
  File "/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/urllib3/connection.py", line 198, in _new_conn
    sock = connection.create_connection(
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/urllib3/util/connection.py", line 85, in create_connection
    raise err
  File "/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/urllib3/util/connection.py", line 73, in create_connection
    sock.connect(sa)
ConnectionRefusedError: [Errno 61] Connection refused

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "/Library/Frameworks/Python.frame

---

* 코드정리

In [ ]:
import gradio as gr
import requests
# from datetime import datetime # 사용되지 않으므로 제거
from openai import AzureOpenAI
import traceback
import json # JSONDecodeError 처리를 위해 추가

# Azure OpenAI 설정
# 보안상 API 키는 코드에 직접 하드코딩하는 것보다 환경 변수나
# 별도의 설정 파일을 사용하는 것이 좋습니다.
# 예: api_key=os.getenv("AZURE_OPENAI_API_KEY")
client = AzureOpenAI(
    api_key="",
    azure_endpoint="https://6b013-azure-ai-service.openai.azure.com/",
    api_version="2024-05-01-preview"
)
AZURE_DEPLOYMENT_NAME = "gpt-4o"

# 카드 업데이트 함수 (사용되지 않으므로 제거)
# def update_cards(title_text):
#     return title_text, title_text, title_text

# 오답 데이터 불러오기
def fetch_wrong_answers_cards(user_id: int = 1) -> str:
    """지정된 사용자의 오답 목록을 API에서 가져와 포맷팅된 문자열로 반환합니다."""
    try:
        # 타임아웃 설정 추가 권장
        res = requests.get("http://localhost:8000/get-wrong-answers", params={"user_id": user_id}, timeout=10)
        res.raise_for_status() # HTTP 오류 발생 시 예외 발생
        data = res.json().get("wrong_answers", []) # 키가 없을 경우 빈 리스트 반환

        if not data:
            return "✅ 오답이 없습니다!"

        cards = []
        for idx, item in enumerate(data, 1):
            # 필수 키 존재 여부 확인 (더 안전한 접근)
            q_text = item.get('question_text', 'N/A')
            c1 = item.get('choice1', 'N/A')
            c2 = item.get('choice2', 'N/A')
            c3 = item.get('choice3', 'N/A')
            c4 = item.get('choice4', 'N/A')
            ans = item.get('answer', 'N/A')
            u_choice = item.get('user_choice', 'N/A')
            exp = item.get('explanation', 'N/A')

            q = f"**{idx}. 문제:** {q_text}"
            choices = (
                f"   - ① {c1}\n"
                f"   - ② {c2}\n"
                f"   - ③ {c3}\n"
                f"   - ④ {c4}"
            )
            answer = f"✅ 정답: {ans}번"
            my_choice = f"❌ 내 선택: {u_choice}번"
            exp_text = f"📘 해설: {exp}"
            cards.append(f"{q}\n{choices}\n{answer} | {my_choice}\n{exp_text}")

        return "\n\n---\n\n".join(cards)
    except requests.exceptions.RequestException as e:
        # 네트워크 관련 오류 처리
        return f"❗ API 연결 오류: {str(e)}"
    except json.JSONDecodeError:
        # JSON 파싱 오류 처리
        return "❗ API 응답 형식 오류 (JSON 파싱 실패)"
    except Exception as e:
        # 기타 예외 처리
        print(f"오답 불러오기 중 예외 발생: {traceback.format_exc()}")
        return f"❗ 오류 발생: {str(e)}"

def update_wrong_note_textbox() -> gr.Textbox:
    """오답 노트 텍스트박스를 최신 오답 데이터로 업데이트합니다."""
    text = fetch_wrong_answers_cards()
    return gr.update(value=text)


# 문제 생성
def add_chat(user_msg: str, history: list | None) -> tuple[list, dict]:
    """사용자 메시지를 받아 AI 모델로 문제를 생성하고, 채팅 기록과 AI 응답 상태를 업데이트합니다."""
    if history is None:
        history = []
    history.append((user_msg, None)) # 사용자 메시지 먼저 추가

    system_prompt = """
너는 한국사능력검정시험 문제를 생성하는 AI 튜터야.
절대로 아래 형식을 벗어나지 마. 다른 문장 추가 금지.

형식 예시:
문제: 백제의 수도가 한성에서 웅진으로 옮겨진 이유는?
보기:
1. 신라와의 동맹
2. 고구려의 침입
3. 내부 반란
4. 일본과의 관계 악화
정답: 2
해설: 475년 고구려 장수왕의 공격으로 백제 수도 한성이 함락되었고, 문주왕은 웅진으로 천도하였다.
"""

    messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_msg}
    ]

    ai_response = {"정답": 0, "해설": "N/A", "question_id": 0} # 기본값 설정

    try:
        completion = client.chat.completions.create(
            model=AZURE_DEPLOYMENT_NAME,
            messages=messages,
            max_tokens=800,
            temperature=0.7
        )

        result = completion.choices[0].message.content
        print("📦 GPT 응답 원문:\n", result) # 디버깅용 출력

        # --- 응답 파싱 ---
        # 이 부분은 LLM 응답 형식 변화에 취약합니다. 정규식 사용이나
        # 더 구조화된 응답 형식을 요청하는 것이 안정적일 수 있습니다.
        if not all(k in result for k in ["문제:", "보기:", "정답:", "해설:"]):
             raise ValueError("GPT 응답 형식이 올바르지 않습니다. (필수 키워드 누락)")

        try:
            question = result.split("문제:")[1].split("보기:")[0].strip()
            choices_block = result.split("보기:")[1].split("정답:")[0].strip()
            choices = [line.strip() for line in choices_block.split('\n') if line.strip()]
            if len(choices) != 4:
                raise ValueError(f"보기 항목이 4개가 아닙니다. (찾은 개수: {len(choices)})")

            answer_str = result.split("정답:")[1].split("해설:")[0].strip()
            answer = int(answer_str) # 정수 변환 시도
            explanation = result.split("해설:")[1].strip()

        except (IndexError, ValueError) as parse_error:
            raise ValueError(f"GPT 응답 파싱 중 오류 발생: {parse_error}") from parse_error
        # --- 파싱 끝 ---

        # --- 문제 저장 API 호출 ---
        save_payload = {
            "material_id": 12, # 이 값은 동적으로 설정될 수 있어야 할 수 있습니다.
            "question_text": question,
            "choice1": choices[0],
            "choice2": choices[1],
            "choice3": choices[2],
            "choice4": choices[3],
            "answer": answer,
            "explanation": explanation
        }
        # 타임아웃 설정 및 오류 처리 강화
        save_res = requests.post("http://localhost:8000/save-question", data=save_payload, timeout=10)
        save_res.raise_for_status() # HTTP 오류 확인
        question_id = save_res.json().get("question_id")
        if question_id is None:
            print("⚠️ 문제 저장 API 응답에 question_id가 없습니다.")
            # 필요시 예외 발생 또는 기본값 처리
        # --- API 호출 끝 ---

        # 성공 시 채팅 기록 및 상태 업데이트
        ai_text = question + "\n" + "\n".join(choices) # 챗봇에 표시될 텍스트
        history.append((None, ai_text)) # AI 응답을 채팅 기록에 추가

        ai_response = {
            "문제": question,
            "보기": choices,
            "정답": answer - 1, # Gradio Radio 인덱스는 0부터 시작하므로 조정
            "해설": explanation,
            "question_id": question_id if question_id is not None else 0 # None일 경우 기본값
        }

    except requests.exceptions.RequestException as e:
        error_msg = f"❌ 문제 저장 API 호출 실패: {str(e)}"
        print(f"🔴 오류: {error_msg}")
        history.append((None, error_msg))
    except (ValueError, IndexError) as e: # 파싱 오류 포함
        error_msg = f"❌ 문제 생성 또는 처리 실패: {str(e)}"
        print(f"🔴 오류: {error_msg}\n{traceback.format_exc()}")
        history.append((None, error_msg))
    except Exception as e: # OpenAI API 오류 등 기타 예외
        error_msg = f"❌ 예상치 못한 오류 발생: {str(e)}"
        print(f"🔴 오류: {error_msg}\n{traceback.format_exc()}")
        history.append((None, error_msg))

    return history, ai_response

# 정답 체크 및 오답 저장 포함
def check_answer(user_choice: str | None, ai_response: dict, history: list | None) -> list:
    """사용자의 선택을 받아 정답을 확인하고, 오답인 경우 저장하며, 채팅 기록을 업데이트합니다."""
    if history is None:
        history = []
    if user_choice is None:
        history.append((None, "⚠️ 정답을 선택해주세요."))
        return history
    if not ai_response or "정답" not in ai_response or "해설" not in ai_response:
         history.append((None, "⚠️ 문제 정보가 올바르지 않아 채점할 수 없습니다."))
         return history

    try:
        user_choice_int = int(user_choice)
        # ai_response["정답"]은 0-based index, user_choice는 1-based string
        correct_answer_display = ai_response["정답"] + 1
        explanation = ai_response["해설"]

        result_msg = f"✅ 정답입니다!" if user_choice_int == correct_answer_display else f"❌ 오답입니다. 정답은 {correct_answer_display}번이에요."

        # 사용자의 선택과 결과를 채팅 기록에 추가
        history.append((f"나의 선택: {user_choice}번", None))
        history.append((None, f"{result_msg}\n📘 해설: {explanation}"))

        # 오답인 경우 저장 시도
        if user_choice_int != correct_answer_display:
            question_id = ai_response.get("question_id")
            if question_id: # question_id가 유효할 때만 저장 시도
                try:
                    # 타임아웃 설정 및 오류 처리 강화
                    wrong_answer_payload = {
                        "user_id": 1, # 이 값도 동적으로 설정될 수 있어야 할 수 있습니다.
                        "question_id": question_id,
                        "user_choice": user_choice_int
                    }
                    res = requests.post("http://localhost:8000/save_wrong_answer", data=wrong_answer_payload, timeout=10)
                    res.raise_for_status() # HTTP 오류 확인
                    print(f"ℹ️ 오답 저장 성공 (Question ID: {question_id})")
                except requests.exceptions.RequestException as e:
                    print(f"❗ 오답 저장 API 호출 실패: {e}")
                except Exception as e:
                    print(f"❗ 오답 저장 중 예상치 못한 오류: {e}")
            else:
                print("⚠️ Question ID가 없어 오답을 저장할 수 없습니다.")

    except ValueError:
        history.append((None, "⚠️ 유효하지 않은 선택입니다."))
    except Exception as e:
        history.append((None, f"⚠️ 채점 중 오류 발생: {str(e)}"))
        print(f"채점 중 오류: {traceback.format_exc()}")

    return history

# 전체 UI 구성
def build_ui() -> gr.Blocks:
    """Gradio 인터페이스를 구성하고 반환합니다."""
    with gr.Blocks(title="한국사능력검정시험 AI 학습") as demo:
        with gr.Tabs():
            # --- AI 문제 탭 ---
            with gr.TabItem("AI 문제"):
                gr.Markdown("### 💬 AI 문제 - RAG 기반 GPT-4o 문제 생성")
                # Chatbot 높이 조정 및 레이아웃 개선 고려
                chatbot = gr.Chatbot(label="한국사 문제 챗봇", height=500, bubble_full_width=False)
                # state: 채팅 기록 (list of tuples)
                state = gr.State([])
                # ai_state: 현재 문제 정보 (dict)
                ai_state = gr.State({})

                with gr.Row():
                    user_input = gr.Textbox(
                        placeholder="예: 백제 문화 관련 문제 내줘",
                        label="문제 요청",
                        scale=8,
                        container=False # 테두리 제거
                    )
                    send_btn = gr.Button("요청", scale=2, variant="primary")

                # 정답 선택 라디오 버튼 (초기에는 숨김)
                answer_dropdown = gr.Radio(
                    choices=["1", "2", "3", "4"],
                    label="정답 선택",
                    visible=False,
                    interactive=True
                )
                # 정답 제출 버튼 (초기에는 숨김)
                submit_answer = gr.Button("정답 제출", visible=False, variant="secondary")

                # --- 이벤트 연결 ---
                # 요청 버튼 클릭 시: add_chat 실행 -> 챗봇, ai_state 업데이트
                send_btn.click(
                    fn=add_chat,
                    inputs=[user_input, state],
                    outputs=[chatbot, ai_state]
                ).then( # add_chat 실행 후
                    # 입력창 비우기
                    lambda: gr.update(value=""), outputs=[user_input]
                ).then(
                    # 정답 선택 및 제출 버튼 표시
                    lambda: (gr.update(visible=True), gr.update(visible=True)),
                    outputs=[answer_dropdown, submit_answer]
                )

                # 엔터 키로 요청 보내기
                user_input.submit(
                    fn=add_chat,
                    inputs=[user_input, state],
                    outputs=[chatbot, ai_state]
                ).then(
                    lambda: gr.update(value=""), outputs=[user_input]
                ).then(
                    lambda: (gr.update(visible=True), gr.update(visible=True)),
                    outputs=[answer_dropdown, submit_answer]
                )

                # 정답 제출 버튼 클릭 시: check_answer 실행 -> 챗봇 업데이트
                submit_answer.click(
                    fn=check_answer,
                    inputs=[answer_dropdown, ai_state, state],
                    outputs=[chatbot]
                ).then( # check_answer 실행 후
                    # 정답 선택 비활성화 및 제출 버튼 숨기기 (새 문제 요청 전까지)
                    lambda: (gr.update(interactive=False), gr.update(visible=False)),
                    outputs=[answer_dropdown, submit_answer]
                )

            # --- 오답 노트 탭 ---
            with gr.TabItem("📕 오답 노트"):
                gr.Markdown("## 📕 나의 오답 노트 (리스트 보기)")
                wrong_note_output = gr.Textbox(
                    lines=20,
                    interactive=False,
                    label="오답 목록",
                    show_copy_button=True
                )
                refresh_btn = gr.Button("🔁 오답 다시 불러오기")

                # 새로고침 버튼 클릭 시: update_wrong_note_textbox 실행 -> 오답 노트 업데이트
                refresh_btn.click(
                    fn=update_wrong_note_textbox,
                    inputs=[],
                    outputs=[wrong_note_output]
                )
                # 탭이 선택될 때 자동으로 오답 노트 로드 (선택 사항)
                # wrong_note_output.attach_load_event(update_wrong_note_textbox, every=None) # Gradio 버전에 따라 다를 수 있음

    return demo

# 애플리케이션 실행
if __name__ == "__main__":
    demo = build_ui()
    # share=True 옵션은 외부 공유 링크 생성 시 사용
    demo.launch()


/var/folders/sh/pq4glsrs3xq2v8ymsrw74mrr0000gn/T/ipykernel_91386/2937927437.py:247: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style dictionaries with 'role' and 'content' keys.
  chatbot = gr.Chatbot(label="한국사 문제 챗봇", height=500, bubble_full_width=False)
/var/folders/sh/pq4glsrs3xq2v8ymsrw74mrr0000gn/T/ipykernel_91386/2937927437.py:247: DeprecationWarning: The 'bubble_full_width' parameter is deprecated and will be removed in a future version. This parameter no longer has any effect.
  chatbot = gr.Chatbot(label="한국사 문제 챗봇", height=500, bubble_full_width=False)


* Running on local URL:  http://127.0.0.1:7863

To create a public link, set `share=True` in `launch()`.


📦 GPT 응답 원문:
 문제: 백제의 건국 시조는 누구인가?  
보기:  
1. 주몽  
2. 온조  
3. 박혁거세  
4. 이성계  
정답: 2  
해설: 백제는 고구려에서 내려온 온조가 한강 유역에서 건국한 나라이다. 온조는 주몽의 아들로 알려져 있다.  
🔴 오류: ❌ 문제 저장 API 호출 실패: HTTPConnectionPool(host='localhost', port=8000): Max retries exceeded with url: /save-question (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x17edff410>: Failed to establish a new connection: [Errno 61] Connection refused'))
